### APIs


#### Weather API 
We will use the Weather API from `visualcrossing`. You have to generate you API key that you can later use to access this API. Follow the steps below: 

1. Signup up at https://www.visualcrossing.com/
2. Verify your account
3. Sign in and click on `Account` (blue button in the top right corner)
4. Under `Details` you should be able to see a `Key`
5. Copy the Key in `helper_functions/keys.py`


#### HuggingFace Token
We will use a model avaialble through a HuggingFace API. For that you need to generate a Token. Follow the steps below: 

1. Visit [HuggingFace](https://huggingface.co/) and sign up or log in.
2. Go to your profile (click your avatar), then "Settings" > "Access Tokens."
3. Click "New Token," select `Fine-grained` as Token Type role, and check the box `Make calls to the serverless Inference API`
4. Copy the Token in `helper_functions/keys.py`.


In [ ]:
import io
import requests
from PIL import Image


# Update helper_functions.keys.py based on private bin link
from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY, OPENAI_KEY

from langchain import hub
from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import AgentExecutor # execute agent
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
# define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="Input city")

# the tool description
description: str = (
        "A wrapper around Weather API. "
        "Useful for when you need to answer weather about "
        "temperature, weather, humidity, rain, sunny, storm, or climate subjects. "
        "Input should be a city."
    )

# fuse the function, input parameters and description into a tool. 
my_weather_tool = StructuredTool.from_function(
    func = extract_city_weather,
    name = "weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False
)
